In [49]:
from Bio import Entrez, SeqIO, AlignIO
import os

In [54]:
# Set your email (made mandatory by NCBI to avoid spams)
Entrez.email = "gunanka.is22@bmsce.ac.in"

In [55]:
def fetch_cox1_sequence(species_name):
    """Fetches the COX1/PTGS1 gene sequence for a given species from NCBI."""

    print(f"Fetching COX1 sequence for '{species_name}' species")
    try:
        # Querying using 3 different terms to make sure we hit
        search_terms = [
            f"{species_name}[Organism] AND (PTGS1[Gene] OR cyclooxygenase-1[Gene Name])",
            f"{species_name}[Organism] AND (COX1[Gene] OR COX-1[Gene] OR PTGS1[Gene])",
            f"{species_name}[Organism] AND cyclooxygenase 1"
        ]
        
        # try all terms
        for term in search_terms:
            print(f"Trying search term: {term}")
            handle = Entrez.esearch(db="nucleotide", term=term, retmax=5)
            record = Entrez.read(handle)
            handle.close()
            
            # if record exists, get its id and sequence
            if record["IdList"]:
                # Fetch the sequence record
                seq_id = record["IdList"][0]
                print(f"SUCCESS : COX1 sequence for {species_name} found. Found ID : {seq_id}")

                handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
                fasta_text = handle.read()
                handle.close()
                
                # print(f"Retrieved sequence with header:\n{fasta_text.split('\\n')[0]}")
                
                # Reopen handle to read with SeqIO
                handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
                seq_record = SeqIO.read(handle, "fasta")
                handle.close()

                print(f"Sequence length: {len(seq_record.seq)} bp\n")

                return seq_record
        
        # None of the terms gave any records
        print(f"No COX1/PTGS1 sequence found for {species_name}")
        return None
    
    except Exception as e:
        print(f"Error fetching sequence: {e}")
        return None

In [ ]:
# testing fetch_cox1_sequence() ...

print("testing fetch_cox1_sequence()...")

# Human beings
species_name = "Homo sapiens" 
fetch_cox1_sequence(species_name)

# Chimpanzees
species_name = "Pan troglodytes" 
fetch_cox1_sequence(species_name)

# Gold fish
species_name = "Carassius auratus" 
fetch_cox1_sequence(species_name)

# True tuna fish
species_name = "Thunnus" 
fetch_cox1_sequence(species_name)

# Yellow fin tuna fish
species_name = "Thunnus albacares" 
fetch_cox1_sequence(species_name)

# Dinosaur ant
species_name = "Nothomyrmecia macrops" 
fetch_cox1_sequence(species_name)

# Odorous house ant
species_name = "Tapinoma sessile" 
fetch_cox1_sequence(species_name)

# Golden eagle
species_name = "Aquila chrysaetos" 
fetch_cox1_sequence(species_name)

# Monarch butterfly
species_name = "Danaus plexippus" 
fetch_cox1_sequence(species_name)

# Sacred Scarab (dung beetle)
species_name = "Scarabaeus sacer" 
fetch_cox1_sequence(species_name)

# Blue Gourami
species_name = "Trichopodus trichopterus" 
fetch_cox1_sequence(species_name)

# Silver Arowana
species_name = "Osteoglossum bicirrhosum" 
fetch_cox1_sequence(species_name)

# Budgerigar
species_name = "Melopsittacus undulatus" 
fetch_cox1_sequence(species_name)

print("testing completed...")

In [56]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def save_sequences_to_fasta(seq1, species_name1, seq2, species_name2, file_path):
    """Saves 2 sequences onto a fasta file in the given file location. Used for MUSCLE"""

    # Create SeqRecord objects
    record1 = SeqRecord(seq1.seq, id="seq1", description=f"{species_name1} COX-1 sequence")
    record2 = SeqRecord(seq2.seq, id="seq2", description=f"{species_name2} COX-1 sequence")
    
    # Write to a FASTA file
    with open(file_path, "w") as output_handle:
        SeqIO.write([record1, record2], output_handle, "fasta")

In [57]:
# testing save_sequences_to_fasta() ...

print("testing save_sequences_to_fasta()...")

# Monarch butterfly
species_name1 = "Danaus plexippus" 
seq1 = fetch_cox1_sequence(species_name)

# Silver Arowana
species_name2 = "Osteoglossum bicirrhosum" 
seq2 = fetch_cox1_sequence(species_name)

file_path = r"../fasta_files/two_cox1_sequences.fasta"

save_sequences_to_fasta(seq1, species_name1, seq2, species_name2, file_path)

print("testing completed...")

testing save_sequences_to_fasta()...
Fetching COX1 sequence for 'Osteoglossum bicirrhosum' species
Trying search term: Osteoglossum bicirrhosum[Organism] AND (PTGS1[Gene] OR cyclooxygenase-1[Gene Name])
Trying search term: Osteoglossum bicirrhosum[Organism] AND (COX1[Gene] OR COX-1[Gene] OR PTGS1[Gene])
SUCCESS : COX1 sequence for Osteoglossum bicirrhosum found. Found ID : 15451817
Sequence length: 16006 bp

Fetching COX1 sequence for 'Osteoglossum bicirrhosum' species
Trying search term: Osteoglossum bicirrhosum[Organism] AND (PTGS1[Gene] OR cyclooxygenase-1[Gene Name])
Trying search term: Osteoglossum bicirrhosum[Organism] AND (COX1[Gene] OR COX-1[Gene] OR PTGS1[Gene])
SUCCESS : COX1 sequence for Osteoglossum bicirrhosum found. Found ID : 15451817
Sequence length: 16006 bp

testing completed...


In [86]:
import subprocess # to use MUSCLE (multiple sequence alignment tool)

def align_sequences_muscle(input_fasta, output_fasta):
    """Aligns sequences using MUSCLE and saves the output to a FASTA file."""
    
    muscle_path = r"../tools/muscle-win64.v5.3.exe"

    # Ensure the output directory exists
    output_dir = os.path.dirname(output_fasta)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

     # For MUSCLE v5.3, the command line syntax has changed
    muscle_cmd = [
        muscle_path, 
        "-align", input_fasta, 
        "-output", output_fasta
    ]
    
    # Full command that is being executed
    print("Running command:", " ".join(muscle_cmd))  

    print("Starting Subprocess")
    result = subprocess.run(muscle_cmd, capture_output=True, text=True)

    # DEBUG : Logs subprocess
    print(f"Result stdout: {result.stdout}")
    print(f"Result stderr: {result.stderr}")

    if result.returncode != 0:
        print(f"Error: MUSCLE failed with exit code {result.returncode}")

In [87]:
# testing align_sequences_muscle() ...

print("testing align_sequences_muscle()...")

input_fasta = r"../fasta_files/two_cox1_sequences.fasta"
output_fasta = r"../fasta_files/aligned_cox1_sequences.fasta"

align_sequences_muscle(input_fasta, output_fasta)

print("testing completed...")

testing align_sequences_muscle()...
Running command: ../tools/muscle-win64.v5.3.exe -align ../fasta_files/two_cox1_sequences.fasta -output ../fasta_files/aligned_cox1_sequences.fasta
Starting Subprocess
Result stdout: 
Result stderr: 
muscle 5.3.win64 [d9725ac]  16.4Gb RAM, 16 cores
Built Nov 10 2024 22:59:05
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

[align ../fasta_files/two_cox1_sequences.fasta]
Input: 2 seqs, avg length 16006, max 16006, min 16006

00:00 1.2Mb    50.0% Derep 1 uniques, 0 dupes
00:00 1.2Mb   100.0% Derep 1 uniques, 1 dupes


testing completed...
